In [ ]:
#RF 5 frame

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
import joblib

# Load the dataset
data = pd.read_csv('Final_new.csv')

# Preprocessing the data
# Create features for the last 5 frames
def create_rolling_features(data, n_frames=5):
    features = []
    labels = []
    
    for i in range(n_frames, len(data)):
        # Get the last n_frames data
        current_data = data.iloc[i-n_frames:i]
        # Flatten the data into a single row
        flat_data = current_data[['Bounding Box Center x', 'Bounding Box Center y', 'Velocity', 'Acceleration', 'Y-Frequency']].values.flatten()
        # Append the label of the current frame
        label = data['Class Label'].iloc[i]
        features.append(flat_data)
        labels.append(label)
    
    return np.array(features), np.array(labels)

# Generate features and labels using the previous 5 frames
X, y = create_rolling_features(data)

# Normalize the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X)

# Apply undersampling to balance the classes
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(scaled_features, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the Random Forest model with balanced data
rf_model = RandomForestClassifier(n_estimators=300, random_state=42, max_depth=30, min_samples_split=5,
                                  min_samples_leaf=2, max_features='sqrt', class_weight='balanced')
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save the model and scaler
joblib.dump(rf_model, 'random_forest_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


Random Forest Model Accuracy: 91.87%
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      3209
           1       0.85      0.98      0.91      3332
           2       0.96      0.81      0.88      3193

    accuracy                           0.92      9734
   macro avg       0.93      0.92      0.92      9734
weighted avg       0.92      0.92      0.92      9734

Confusion Matrix:
[[3090   52   67]
 [  29 3256   47]
 [  73  523 2597]]


['scaler.pkl']

In [ ]:
import cv2
import numpy as np
import joblib
import pandas as pd
from ultralytics import YOLO

# Constants
MAX_DISTANCE = 100
CONFIDENCE_THRESHOLD = 0.5
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.8
FONT_COLOR = (0, 255, 0)  # Green for text
FONT_THICKNESS = 2
LINE_COLOR = (0, 0, 255)  # Red for tracking lines
SEQ_LENGTH = 5  # Length of sequence for LSTM (not needed for RF)

# Load models
yolo_model = YOLO("best.pt")
rf_model = joblib.load('random_forest_model.pkl')  # Load the Random Forest model
scaler = joblib.load('scaler.pkl')  # Load the scaler for feature normalization

# Initialize tracking variables
next_id = 0
tracked_objects = {}
object_paths = {}
object_velocities = {}
object_accelerations = {}
object_frequencies = {}
frame_data = {}  # Dictionary to store features for each object over frames

# Path to the video file
video_path = 'D:/Segment/Drones/D12.mp4'  # Update with your video file path
cap = cv2.VideoCapture(video_path)  # Load the video file
if not cap.isOpened():
    raise ValueError("Error: Cannot open video file")

# Get frame rate for calculations
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_time = 1 / fps if fps > 0 else 1 / 30  # Time between frames in seconds (default to 30 FPS if unavailable)

def euclidean_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def associate_detections(tracked_objects, current_frame_objects):
    updated_tracked_objects = {}
    unmatched_current_objects = current_frame_objects.copy()

    for obj_id, data in tracked_objects.items():
        prev_cx, prev_cy, prev_x, prev_y, prev_w, prev_h = data
        best_match = None
        min_distance = float('inf')

        for (cx, cy, x, y, w, h, class_id) in unmatched_current_objects:
            distance = euclidean_distance((cx, cy), (prev_cx, prev_cy))
            if distance < min_distance and distance < MAX_DISTANCE:
                min_distance = distance
                best_match = (cx, cy, x, y, w, h, class_id)

        if best_match:
            updated_tracked_objects[obj_id] = best_match[:6]
            unmatched_current_objects.remove(best_match)
            object_paths[obj_id].append(best_match[:2])

            # Calculate velocity
            prev_pos = np.array((prev_cx, prev_cy))
            curr_pos = np.array(best_match[:2])
            velocity_vector = (curr_pos - prev_pos) / frame_time
            velocity_magnitude = np.linalg.norm(velocity_vector)
            object_velocities[obj_id].append(velocity_magnitude)

            # Calculate acceleration
            if len(object_velocities[obj_id]) > 1:
                acceleration = (object_velocities[obj_id][-1] - object_velocities[obj_id][-2]) / frame_time
            else:
                acceleration = 0
            object_accelerations[obj_id].append(acceleration)

            # Calculate y-frequency
            y_changes = [abs(object_paths[obj_id][i][1] - object_paths[obj_id][i - 1][1]) 
                        for i in range(1, len(object_paths[obj_id]))]
            frequency = len([change for change in y_changes if change > 0]) / frame_time
            object_frequencies[obj_id].append(frequency)

        else:
            updated_tracked_objects[obj_id] = data

    return updated_tracked_objects, unmatched_current_objects

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detection with YOLO model
    results = yolo_model(frame)
    current_frame_objects = []
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
            w, h = x2 - x1, y2 - y1
            class_id = int(box.cls[0])
            confidence = box.conf[0]
            if confidence > CONFIDENCE_THRESHOLD:
                current_frame_objects.append((cx, cy, int(x1), int(y1), int(w), int(h), class_id))

    tracked_objects, unmatched_current_objects = associate_detections(tracked_objects, current_frame_objects)

    # Draw tracking lines, bounding boxes, and display object data
    for obj_id, (cx, cy, x, y, w, h) in tracked_objects.items():
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Prepare features for Random Forest model
        velocity = object_velocities[obj_id][-1] if object_velocities[obj_id] else 0
        acceleration = object_accelerations[obj_id][-1] if object_accelerations[obj_id] else 0
        y_frequency = object_frequencies[obj_id][-1] if object_frequencies[obj_id] else 0
        
        # Collect frame data for prediction
        feature_vector = np.array([[cx, cy, velocity, acceleration, y_frequency]])
        feature_vector = scaler.transform(feature_vector)  # Normalize features

        # Perform prediction using Random Forest model
        predicted_class = rf_model.predict(feature_vector)[0]

        # Mapping predicted class to string labels
        class_label = ''
        if predicted_class == 0:  # Assuming class labels are integers starting from 0
            class_label = 'drone'
        elif predicted_class == 1:
            class_label = 'eagle'
        elif predicted_class == 2:
            class_label = 'bird'

        # Display classification label
        object_info = f'ID: {obj_id} Class: {class_label}'
        cv2.putText(frame, object_info, (x, y - 15), FONT, FONT_SCALE, FONT_COLOR, FONT_THICKNESS)

        # Draw tracking line
        if obj_id in object_paths and len(object_paths[obj_id]) > 1:
            for i in range(1, len(object_paths[obj_id])):
                cv2.line(frame, object_paths[obj_id][i - 1], object_paths[obj_id][i], LINE_COLOR, 2)

    for (cx, cy, x, y, w, h, class_id) in unmatched_current_objects:
        tracked_objects[next_id] = (cx, cy, x, y, w, h)
        object_paths[next_id] = [(cx, cy)]
        object_velocities[next_id] = [0]
        object_accelerations[next_id] = [0]
        object_frequencies[next_id] = [0]
        frame_data[next_id] = []
        next_id += 1

    # Display the frame
    cv2.imshow("Real-Time Detection", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
